In [ ]:
import Ipynb_importer
from window_login import WindowLogin
from request_protocol import RequestProtocol
from client_socket import ClientSocket
from threading import Thread
from config import *
from tkinter.messagebox import showinfo
from window_chat import WindowChat
import os

class Client(object):  # 客户端核心
    def __init__(self):  # 初始化客户端资源
        # 初始化登录窗口
        self.window = WindowLogin()
        self.window.reset_button_click(self.clear_inputs)
        self.window.login_button_click(self.send_login_data)
        self.window.window_close(self.exit)  # 关闭窗口时退出程序
        
        # 初始化聊天窗口
        self.window_chat = WindowChat()
        self.window_chat.send_button_click(self.send_chat_data)
        self.window_chat.withdraw()  # 隐藏窗口
        self.window_chat.window_close(self.exit)  # 关闭窗口时退出程序
        
        self.soc = ClientSocket()  # 创建客户端套接字
        
        # 初始化消息处理函数
        self.response_handle_function = {}
        self.register(RESPONSE_LOGIN_RESULT, self.response_login_handle)
        self.register(RESPONSE_CHAT, self.response_chat_handle)
        self.register(RESPONSE_TIME, self.response_time_handle)
        self.register(RESPONSE_FILE, self.response_file_handle)
        self.register(RESPONSE_SEND, self.response_send_handle)
        
        self.username = None  # 登录用户账号
        self.self = None  # 登录用户昵称
        
        self.is_running = True  # 程序正在运行的flag
        
        self.file = None  # 正在发送的文件
        self.is_receiving = False  # 程序正在接收文件的flag
        self.file_size = None  # 文件大小
        self.recv_size = None  # 已接收的文件大小
        
    def register(self, request_id, handle_function):  # 注册响应和对应的方法到字典中
        self.response_handle_function[request_id] = handle_function
        
    def startup(self):  # 开启窗口
        self.soc.connect()
        Thread(target = self.response_handle).start()  # 在mainloop前开启接收消息的子线程
        self.window.mainloop()
        
    def clear_inputs(self):  # 清空文本框内容
        self.window.clear_username()
        self.window.clear_password()
        
    def send_login_data(self):  # 发送登录消息到服务器
        # 获取输入的账号和密码
        username = self.window.get_username()
        password = self.window.get_password()
        
        # 发送登录消息到服务器
        request_text = RequestProtocol.request_login_result(username, password)
        self.soc.send_data(request_text)
        
        # 接收登录结果
        # recv_data = self.soc.recv_data()
        # print(recv_data)
        
    def send_chat_data(self):  # 发送聊天消息到服务器
        message = self.window_chat.get_inputs()
        self.window_chat.clear_inputs()  # 清空文本框
        
        self.window_chat.append_message(self.nickname, message, "user_self")  # 将消息添加到聊天区
        
        if message[0:5] == "/time":  # 获取服务器时间请求
            request_text = RequestProtocol.request_time(self.username)  # 拼接协议文本
            self.soc.send_data(request_text)
        elif message[0:5] == "/file":  # 获取服务器文件请求
            filename = message[6:]
            request_text = RequestProtocol.request_file(self.username, filename)  # 拼接协议文本
            self.soc.send_data(request_text)
        else:  # 聊天请求
            request_text = RequestProtocol.request_chat(self.username, message)  # 拼接协议文本
            self.soc.send_data(request_text)
    
    def response_handle(self):  # 接收服务器消息
        while self.is_running:
            try:
                recv_data = self.soc.recv_data()  # 接收服务器消息
                print("A message was got: " + recv_data)
                response_data = self.parse_response_data(recv_data)  # 解析消息

                # 根据响应类型分别处理
                handle_function = self.response_handle_function[response_data["response_id"]]
                if handle_function:
                    handle_function(response_data)
            except:
                break
   
    @staticmethod
    def parse_response_data(recv_data):  # 解析消息
        '''
        登录响应: 1001|0/1|nickname|username
        聊天响应: 1002|nickname|message
        获取服务器时间响应: 1003|message
        获取服务器文件响应: 1004|filesize
        服务器发送文件响应: 1005|data
        '''
        response_data_list = recv_data.split(DELIMITER)  # 按分隔符分割消息
        
        # 解析消息的各部分
        response_data = {}
        response_data["response_id"] = response_data_list[0]
        
        if response_data["response_id"] == RESPONSE_LOGIN_RESULT:  # 登录响应
            response_data["result"] = response_data_list[1]
            response_data["nickname"] = response_data_list[2]
            response_data["username"] = response_data_list[3]
        elif response_data["response_id"] == RESPONSE_CHAT:  # 聊天响应
            response_data["nickname"] = response_data_list[1]
            response_data["message"] = response_data_list[2]
        elif response_data["response_id"] == RESPONSE_TIME:  # 获取服务器时间响应
            response_data["message"] = response_data_list[1]
        elif response_data["response_id"] == RESPONSE_FILE:  # 获取服务器文件响应
            response_data["filename"] = response_data_list[1]
            response_data["filesize"] = response_data_list[2]
        elif response_data["response_id"] == RESPONSE_SEND:  # 服务器发送文件响应
            response_data["data"] = response_data_list[1]
        
        return response_data
        
    def response_login_handle(self, response_data):  # 登录响应
        # print("A login answer received", response_data)
        result = response_data["result"]
        
        # 登录失败
        if result == "0":
            showinfo("Login failed.", "Account or password error!")
            return
        
        # 登录成功
        self.nickname = response_data["nickname"]  # 保存登录用户的昵称, 供将消息添加到聊天区使用
        self.username = response_data["username"]  # 保存登录用户的账号, 供发送消息使用
        showinfo("Login successfully.", "Hello " + self.nickname + "!")
        
        # 显示聊天窗口
        self.window_chat.set_title(self.username)
        self.window_chat.update()  # 刷新窗口内容
        self.window_chat.deiconify()
        
        self.window.withdraw()  # 隐藏登录窗口
    
    def response_chat_handle(self, response_data):  # 聊天响应
        # print("A chat answer received", response_data)
        sender = response_data["nickname"]
        message = response_data["message"]
        self.window_chat.append_message(sender, message, "user_others")
        
    def response_time_handle(self, response_data):  # 获取服务器时间响应
        sender = "System"
        message = response_data["message"] + "\n"  # 服务器发来的消息最后无\n
        self.window_chat.append_message(sender, message, "system")
        
    def response_file_handle(self, response_data):  # 获取服务器文件响应
        filename = response_data["filename"]
        filename = "received_" + filename  # 保存文件的名称
        filepath = os.path.join(SAVE_PATH, filename)  # 保存文件的路径
        self.file = open(filepath, "wb")  # 以二进制写的形式打开文件
        self.is_receiving = True  # 文件接收flag
        
        self.file_size = int(response_data["filesize"])
        self.recv_size = 0  # 已接收的文件大小
        
        print("Start to receive ...")
        
    def response_send_handle(self, response_data):
        data = response_data["data"]
        data = bytes(data, encoding = "gbk")  # 防止中文乱码
        # data = response_data["data"].decode("utf-8")
        if self.recv_size + SEND_CHUNK >= self.file_size:  # 最后一批
            print("Received a chunk of %d Bytes." % (self.file_size - self.recv_size))
            self.recv_size = self.file_size
        else:  # 非最后一批
            print("Received a chunk of %d Bytes." % SEND_CHUNK)
            self.recv_size += SEND_CHUNK
        
        print(self.recv_size, self.file_size)
        
        # 以二进制写入文件
        try:
            self.file.write(data)
        finally:
            if self.recv_size >= self.file_size:  # 文件接收完毕
                self.is_receiving = False
                # self.file.close()
                print("Received the file %s (%d Bytes) sucessfully." % (self.filename, self.file_size))
        
#         if self.recv_size >= self.file_size:  # 文件接收完毕
#             self.is_receiving = False
#             self.file.close()
#             print("Received the file %s (%d Bytes) sucessfully." % (self.filename, self.file_size))
        
    def exit(self):  # 释放资源并退出程序
        self.is_running = False  # 停止子线程
        self.soc.close()  # 关闭套接字
        os._exit(0)  # 退出程序

if __name__ == "__main__":
    client = Client()
    client.startup()

A message was got: 1001|0|user1|
A message was got: 1001|1|konbi|user1
A message was got: 1002|Hytidel|Hello!

A message was got: 1003|The server time is 2023-04-06 12:33:11


In [ ]:
print(self.file, self.recv_size, self.file_size)

In [6]:
from threading import Thread
import time

def func1():
    for i in range(5):
        print("func1 was called.")
        time.sleep(0.5)

def func2():
    for i in range(5):
        print("func2 was called.")
    
th1 = Thread(target = func1)
th1.start()

th2 = Thread(target = func2)
th2.start()
th2.join()


func1 was called.
func2 was called.
func2 was called.
func2 was called.
func2 was called.
func2 was called.
func1 was called.
func1 was called.
func1 was called.
func1 was called.
